In [ ]:
import huggingface_hub
hf_token = '...' # put your User Access Tokens here
# ابتدا login کنید
huggingface_hub.login(token=hf_token)

# سپس وضعیت ورود را بررسی کنید
!hf auth whoami

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


user:  AM-Nateghi


In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training
import torch

cptk = "google/gemma-3-1b-pt"
from transformers import BitsAndBytesConfig
tokenizer = AutoTokenizer.from_pretrained(cptk)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
model = AutoModelForCausalLM.from_pretrained(
    cptk,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.float16,
)
model = prepare_model_for_kbit_training(model)

In [20]:
from peft import LoraConfig, get_peft_model, TaskType

lora_conf = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias='none',
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_conf)

In [21]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={
    "train": "assets/qa_train.json",
    "test": "assets/qa_test.json"
})

def formatter(example):
    return tokenizer(
        f"پرسش: {example['input']}\nپاسخ: {example['output']}\n",
        truncation=True,
        max_length=10000,
        padding="max_length"
    )

tokenized_dataset = dataset.map(formatter)

Map: 100%|██████████| 48/48 [00:00<00:00, 328.47 examples/s]


In [28]:
torch.cuda.empty_cache()

In [30]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback

callbacks = [EarlyStoppingCallback(early_stopping_patience=8)]

training_args = TrainingArguments(
    output_dir="./gemma_qlora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    save_steps=100,
    save_total_limit=2,
    report_to="none",
    metric_for_best_model="loss",
    save_strategy='steps',
    load_best_model_at_end=True,
    eval_strategy="steps",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
    processing_class=None,
    callbacks=callbacks,
)
trainer.tokenizer = tokenizer
trainer.train()

Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.
/home/ai/Desktop/Fine-Tune/.env/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 8.00 GiB. GPU 0 has a total capacity of 11.59 GiB of which 4.52 GiB is free. Including non-PyTorch memory, this process has 6.63 GiB memory in use. Of the allocated memory 5.61 GiB is allocated by PyTorch, and 812.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)